In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import math
import altair as alt
import folium
import matplotlib
import matplotlib.pyplot as plt
from folium import IFrame
from numpy import linalg
import scipy
import branca.colormap as cm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np
import itertools

# enable correct rendering
alt.renderers.enable('default')
# uses intermediate json files to speed things up
alt.data_transformers.enable('json')


DataTransformerRegistry.enable('json')

In [ ]:
#Importing Main Dataset
df = pd.read_csv('data2.csv', encoding='latin1')

In [ ]:

#Dataframe Cleanup
#Stockcode which contains only digits signifies sale entry
#Hence we will filter out enteries with digit only stockcode

#Carriage did not drop, also look for returns if the qt. is negative and if they have same invoiceid 

clean_df = df[df.StockCode.str.contains('^\d', regex=True, na=False)]

# Drop quantities which are negative 
clean_df = clean_df[clean_df["Quantity"] >= 0]

#Adding SalesValue column to the Dataframe
clean_df["SalesValue"] = clean_df["Quantity"]*clean_df["UnitPrice"]

#UK Dataframe
uk_df = clean_df[clean_df['Country'] == 'United Kingdom']

#Rest of the World Dataframe
row_df = clean_df.drop(clean_df[clean_df['Country'] == 'United Kingdom'].index)

In [ ]:
#Grouping sales by customers
customer_df = clean_df.groupby(by=['CustomerID']).agg({'Quantity':['min', 'max', 'sum'], 'UnitPrice':['min', 'max', 'sum']})
print("Number of unique customers = ", customer_df.index.size)


Number of unique customers =  4335


In [ ]:
# Importing Secondary Datasets - Records for year range 2007 to 2011
# HDI could be additional variable
# Smartphone penetration dataset if available
# Credit card 

#Purchasing power parity GDP, PPP (constant 2017 international $) | Data (worldbank.org)
gdp_df = pd.read_excel('API_NY.GDP.MKTP.PP.KD_DS2_en_excel_v2_2764839.xls', 
                       sheet_name=0, header=3, usecols="A,B,D,AZ:BD")

#Inflation CPI Consumer price index (2010 = 100) | Data (worldbank.org)
cpi_df = pd.read_excel("API_FP.CPI.TOTL_DS2_en_excel_v2_2765329.xls",
                       sheet_name=0, header=3, usecols="A,B,D,AZ:BD")

#Debt % versus GDP External debt stocks, long-term (DOD, current US$) | Data (worldbank.org)
#extdebt_df = pd.read_excel("API_DT.DOD.DLXF.CD_DS2_en_excel_v2_2823747.xls",
#                           sheet_name=0, header=3, usecols="A,B,D,AZ:BD")

#Individuals using the Internet (% of population) 

internet_df = pd.read_excel("API_IT.NET.USER.ZS_DS2_en_excel_v2_2764008.xls",
                    sheet_name=0, header=3, usecols="A,B,D,AZ:BD")

#Exchange rate fluctuation (L5Y) Official exchange rate (LCU per US$, period average) | Data (worldbank.org)
exchrate_df = pd.read_excel("API_PA.NUS.FCRF_DS2_en_excel_v2_2764464.xls",
                            sheet_name=0, header=3, usecols="A,B,D,AZ:BD")

#Population Population, total | Data (worldbank.org)
pop_df = pd.read_excel("API_SP.POP.TOTL_DS2_en_excel_v2_2764317.xls",
                       sheet_name=0, header=3, usecols="A,B,D,AZ:BD")

#Merchandise imports Merchandise imports (current US$) | Data (worldbank.org)
merch_df = pd.read_excel("API_TM.VAL.MRCH.CD.WT_DS2_en_excel_v2_2766285.xls",
                         sheet_name=0, header=3, usecols="A,B,D,AZ:BD")
## new dataset
expendHealth_df = pd.read_excel("Expenditure_on_health.xls",
                         sheet_name=0, header=3, usecols="A,B,D,AZ:BD")
## new dataset
lifeExpect_df = pd.read_excel("life_expectancy.xls",
                         sheet_name=0, header=3, usecols="A,B,D,AZ:BD")

## new dataset
PPP_per_capita_df = pd.read_excel("PPP_per_capita.xls",
                         sheet_name=0, header=3, usecols="A,B,D,AZ:BD")

#CERDI Sea Distance Dataset
seadist_df = pd.read_excel("CERDI-seadistance.xlsx", usecols="A,B,C")
# Cleaning Sea Distance DF to only include entries with UK as the origin
seadist_df = seadist_df[seadist_df["iso1"]=="GBR"]


In [ ]:
# Function to normalize country names to code that will be used as a key to combine all datasets
def valeurs(k):
    filtered={'United Kingdom': 'GBR',
 'France': 'FRA',
 'USA': 'USA',
 'Belgium': 'BEL',
 'Australia': 'AUS',
 'EIRE': 'IRL',
 'Germany': 'DEU',
 'Portugal': 'PRT',
 'Japan': 'JPN',
 'Denmark': 'DNK',
 'Nigeria': 'NGA',
 'Netherlands': 'NLD',
 'Poland': 'POL',
 'Spain': 'ESP',
 'Channel Islands': 'CHI',
 'Italy': 'ITA',
 'Cyprus': 'CYP',
 'Greece': 'GRC',
 'Norway': 'NOR',
 'Austria': 'AUT',
 'Sweden': 'SWE',
 'United Arab Emirates': 'ARE',
 'Finland': 'FIN',
 'Switzerland': 'CHE',
 'Malta': 'MLT',
 'Bahrain': 'BHR',
 'Bermuda': 'BMU',
 'Hong Kong': 'HKG',
 'Singapore': 'SGP',
 'Thailand': 'THA',
 'Israel': 'ISR',
 'Lithuania': 'LTU',
 'Lebanon': 'LBN',
 'Korea': 'KOR',
 'Brazil': 'BRA',
 'Canada': 'CAN',
 'Iceland': 'ISL'}
    try:
        x=filtered[k]
    except:
        x=None
    return x

In [ ]:

# Function to get all the country related information for the analysis task
def countrydf(name, frequency="M"):
    # Country specific local dataframe
    cdf = clean_df.loc[clean_df["Country"] == name]
    # Datetime conversion
    cdf["date"] = pd.to_datetime(cdf.InvoiceDate)
    # Now lets find cummulative Sales for plot
    # First lets group by Date to get transaction total per day
    plotdf = cdf.set_index("date").resample(frequency)['SalesValue'].sum()
    # Convert series to dataframe
    plotdf = plotdf.to_frame()
    # Total sales for the country
    sales = plotdf["SalesValue"].sum()
    # DF that conforms to 13 row format if frequency is set to monthly
    if frequency == "M":
        plotdf = (plotdf + dummydf).fillna(0)
    # Number of unique customers in the country
    custcnt = len(cdf["CustomerID"].unique())
    # Numpy array of unique stock sold in each country
    uniquestock = cdf["StockCode"].unique() #add .tolist() if list output is desired
    #Country code
    code = valeurs(name)
    #GDP, CPI,  for year 2011
    gdp = gdp_df.loc[gdp_df["Country Code"] == code, "2011"].item()
    cpi = cpi_df.loc[cpi_df["Country Code"] == code, "2011"].item()
    #exchrate = exchrate_df.loc[exchrate_df["Country Code"] == code, "2011"].item()
    pop = pop_df.loc[pop_df["Country Code"] == code, "2011"].item()
    merch = merch_df.loc[merch_df["Country Code"] == code, "2011"].item()
    internet = internet_df.loc[internet_df["Country Code"] == code, "2011"].item()
    life_expect=lifeExpect_df.loc[internet_df["Country Code"] == code, "2011"].item()
    
    expend_health = expendHealth_df.loc[internet_df["Country Code"] == code, "2011"].item()
    ppp_capita = PPP_per_capita_df.loc[internet_df["Country Code"] == code, "2011"].item()

    
    
    #Sea Distance
    dist = seadist_df.loc[seadist_df["iso2"]==code, "seadistance"].item()
    
    return {'name':name, 'code':code, 'df':plotdf, 'totalsales':sales, 'customercnt':custcnt, 
            'uniqueStockID': uniquestock, 'gdp':gdp, 'cpi':cpi, 'population':pop, 
            'merchsales': merch, 'internet':internet, 'distance':dist, 'expend_health':expend_health,'ppp_cap':ppp_capita,
            'life_expect':life_expect}


# Creating dummydf to obtain fixed 13 row plotdf
dummyindex = ['2010-12-31', '2011-01-31', '2011-02-28', '2011-03-31',
             '2011-04-30', '2011-05-31', '2011-06-30', '2011-07-31',
             '2011-08-31', '2011-09-30', '2011-10-31', '2011-11-30',
               '2011-12-31']
dummyvalues = [0,0,0,0,0,0,0,0,0,0,0,0,0]
dummydf = pd.DataFrame({'date':dummyindex, 'SalesValue':dummyvalues})
dummydf = dummydf.set_index('date')


In [ ]:
# Creating Final DF that will be used for regression analysis
# Safe to ignore the SettingWithCopyWarning warning
countries = ['Australia','France', 'USA', 'Belgium', 'EIRE', 'Germany', 'Portugal', 'Japan', 'Denmark', 'Nigeria', \
    'Netherlands', 'Poland', 'Spain', 'Italy', 'Cyprus', 'Greece','Norway', 'Austria', 'Sweden', \
        'United Arab Emirates', 'Finland', 'Switzerland', 'Malta', 'Bahrain', 'Bermuda', 'Hong Kong', \
            'Singapore', 'Thailand', 'Israel', 'Lithuania', 'Lebanon', 'Korea', 'Brazil', 'Canada', 'Iceland']
# Creating list of dictionaries obtained using countrydf function
finallist = [countrydf(country) for country in countries]
# Creating Dataframe from that list
finaldf = pd.DataFrame(finallist)
finaldf.head(3)

liste={'France': 'FRA',
 'USA': 'USA',
 'Belgium': 'BEL',
 'Australia': 'AUS',
 'EIRE': 'IRL',
 'Germany': 'DEU',
 'Portugal': 'PRT',
 'Japan': 'JPN',
 'Denmark': 'DNK',
 'Nigeria': 'NGA',
 'Netherlands': 'NLD',
 'Poland': 'POL',
 'Spain': 'ESP',
 'Channel Islands': 'CHI',
 'Italy': 'ITA',
 'Cyprus': 'CYP',
 'Greece': 'GRC',
 'Norway': 'NOR',
 'Austria': 'AUT',
 'Sweden': 'SWE',
 'United Arab Emirates': 'ARE',
 'Finland': 'FIN',
 'Switzerland': 'CHE',
 'Unspecified': '',
 'Malta': 'MLT',
 'Bahrain': 'BHR',
 'RSA': '',
 'Bermuda': 'BMU',
 'Hong Kong': 'HKG',
 'Singapore': 'SGP',
 'Thailand': 'THA',
 'Israel': 'ISR',
 'Lithuania': 'LTU',
 'West Indies': '',
 'Lebanon': 'LBN',
 'Korea': 'KOR',
 'Brazil': 'BRA',
 'Canada': 'CAN',
 'Iceland': 'ISL'}
liste=list(liste.keys())

set(countries) ^ set(liste)

<ipython-input-55-5aa9c555ed1d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdf["date"] = pd.to_datetime(cdf.InvoiceDate)


{'Channel Islands', 'RSA', 'Unspecified', 'West Indies'}

In [ ]:
finaldf.columns

Index(['name', 'code', 'df', 'totalsales', 'customercnt', 'uniqueStockID',
       'gdp', 'cpi', 'population', 'merchsales', 'internet', 'distance',
       'expend_health', 'ppp_cap', 'life_expect'],
      dtype='object')

In [ ]:
sales=finaldf.df
sales

0                 SalesValue
date                  
...
1                 SalesValue
date                  
...
2                 SalesValue
date                  
...
3                 SalesValue
date                  
...
4                 SalesValue
date                  
...
5                 SalesValue
date                  
...
6                 SalesValue
date                  
...
7                 SalesValue
date                  
...
8                 SalesValue
date                  
...
9                 SalesValue
date                  
...
10                SalesValue
date                  
...
11                SalesValue
date                  
...
12                SalesValue
date                  
...
13                SalesValue
date                  
...
14                SalesValue
date                  
...
15                SalesValue
date                  
...
16                SalesValue
date                  
...
17                SalesValue
date               

In [ ]:
#Example of how to extract Monthly Sales data for each country
# In this example, we will use Canada and for that code is "CAN"
example = finaldf.loc[finaldf["code"]=='CAN', 'df'].item()
example 

,SalesValue
date,
2010-12-31,0.00
2011-01-31,0.00
2011-02-28,0.00
2011-03-31,140.54
2011-04-30,0.00
2011-05-31,534.24
2011-06-30,1171.46
2011-07-31,1217.64
2011-08-31,51.56


In [ ]:
# cols=table.columns.difference(['count'])
# cols=['2009-12', '2010-01', '2010-02', '2010-03', '2010-04', '2010-05',
#       '2010-06', '2010-07', '2010-08', '2010-09', '2010-10', '2010-11',
#       '2010-12']

In [ ]:
##table['count'] = table[table[cols] > 0].count(axis=1)
#table_short=table[table['count']>=7].copy()

# convert negative values to zero
#table_short[table_short < 0] = 0
# table_short.astype(int)

In [ ]:
### EXPAND sales by month for each country. 
## add a column that 

table

,name,df,totalsales
0,Australia,SalesValue date ...,138171.31
1,France,SalesValue date ...,184582.74
2,USA,SalesValue date ...,3580.39
3,Belgium,SalesValue date ...,36927.34
4,EIRE,SalesValue date ...,271164.30
5,Germany,SalesValue date ...,205569.89
6,Portugal,SalesValue date ...,27015.16
7,Japan,SalesValue date ...,37416.37
8,Denmark,SalesValue date ...,18211.34
9,Nigeria,SalesValue date ...,0.00


In [ ]:
finaldf.head()

,name,code,df,totalsales,customercnt,uniqueStockID,gdp,cpi,population,merchsales,internet,distance,expend_health,ppp_cap,life_expect
0,Australia,AUS,SalesValue date ...,138171.31,9,"[22941, 21622, 21791, 35004C, 35004G, 85014B, ...",1.015660e+12,103.303850,22340024.0,2.437010e+11,79.487698,20225.200,4.508153,13905.245764,71.234
1,France,FRA,SalesValue date ...,184582.74,88,"[22728, 22727, 22726, 21724, 21883, 10002, 217...",2.800855e+12,102.111598,65342789.0,7.200280e+11,77.819999,3224.950,13.095275,3765.150290,66.646
2,USA,USA,SalesValue date ...,3580.39,4,"[22722, 22979, 84987, 22720, 22993, 47580, 229...",1.706195e+13,103.156842,311583481.0,2.266024e+12,69.729461,7767.240,4.465111,11393.574239,71.827
3,Belgium,BEL,SalesValue date ...,36927.34,25,"[84375, 21217, 21212, 21977, 22417, 21975, 219...",5.315359e+11,103.532082,11038264.0,4.669430e+11,81.609996,323.696,5.229733,1784.619497,57.761
4,EIRE,IRL,SalesValue date ...,271164.30,4,"[22968, 85071A, 85071C, 22355, 21579, 21576, 2...",2.464671e+11,102.557189,4580084.0,6.660600e+10,74.889973,734.066,2.791318,9435.590356,68.848


,name,df,totalsales
0,Australia,SalesValue date ...,138171.31
1,France,SalesValue date ...,184582.74
2,USA,SalesValue date ...,3580.39
3,Belgium,SalesValue date ...,36927.34
4,EIRE,SalesValue date ...,271164.30
5,Germany,SalesValue date ...,205569.89
6,Portugal,SalesValue date ...,27015.16
7,Japan,SalesValue date ...,37416.37
8,Denmark,SalesValue date ...,18211.34
9,Nigeria,SalesValue date ...,0.00


In [ ]:
finaldf.columns

Index(['name', 'code', 'df', 'totalsales', 'customercnt', 'uniqueStockID',
       'gdp', 'cpi', 'population', 'merchsales', 'internet', 'distance',
       'expend_health', 'ppp_cap', 'life_expect'],
      dtype='object')

In [ ]:
# Establishing factors list
liste_factores=['gdp', 'cpi', 'population', 'merchsales', 'internet', 'distance',
       'expend_health', 'ppp_cap', 'life_expect']
# liste_factores.append('distance')
print(liste_factores)

['gdp', 'cpi', 'population', 'merchsales', 'internet', 'distance', 'expend_health', 'ppp_cap', 'life_expect']


In [ ]:
da=finaldf[['name','totalsales','gdp', 'cpi', 'population', 'merchsales', 'internet', 'distance',
       'expend_health', 'ppp_cap', 'life_expect']]

In [ ]:
corr=da[['totalsales','gdp', 'cpi', 'population', 'merchsales', 'internet', 'distance',
       'expend_health', 'ppp_cap', 'life_expect']].corr().round(2)
corr.style.background_gradient(cmap='coolwarm')

,totalsales,gdp,cpi,population,merchsales,internet,distance,expend_health,ppp_cap,life_expect
totalsales,1.000000,0.020000,-0.250000,-0.060000,0.220000,0.280000,-0.240000,-0.220000,-0.270000,-0.190000
gdp,0.020000,1.000000,-0.070000,0.860000,0.910000,-0.030000,0.130000,-0.210000,-0.060000,-0.050000
cpi,-0.250000,-0.070000,1.000000,0.280000,-0.150000,-0.630000,0.060000,0.010000,0.410000,0.060000
population,-0.060000,0.860000,0.280000,1.000000,0.720000,-0.370000,0.210000,-0.240000,0.220000,0.040000
merchsales,0.220000,0.910000,-0.150000,0.720000,1.000000,0.110000,0.130000,-0.160000,-0.210000,-0.250000
internet,0.280000,-0.030000,-0.630000,-0.370000,0.110000,1.000000,-0.200000,0.100000,-0.200000,-0.060000
distance,-0.240000,0.130000,0.060000,0.210000,0.130000,-0.200000,1.000000,-0.020000,-0.130000,-0.290000
expend_health,-0.220000,-0.210000,0.010000,-0.240000,-0.160000,0.100000,-0.020000,1.000000,0.160000,0.090000
ppp_cap,-0.270000,-0.060000,0.410000,0.220000,-0.210000,-0.200000,-0.130000,0.160000,1.000000,0.710000
life_expect,-0.190000,-0.050000,0.060000,0.040000,-0.250000,-0.060000,-0.290000,0.090000,0.710000,1.000000


In [ ]:
import seaborn as sns
import altair as alt
## scatter plot or SPLOMS

chart = alt.Chart(da).mark_point(fill='red',fillOpacity=0.6,size=70).encode(
    x=alt.X('totalsales:Q',axis=alt.Axis(labelFontSize=12,titleFontSize=14,title="totalsales")),
    y=alt.Y('expend_health:Q',axis=alt.Axis(labelFontSize=12,titleFontSize=14,title='expend_health'))
)
chart+chart.transform_regression('totalsales', 'expend_health').mark_line(line=True,fill='black',strokeDash=[1,5],stroke='black',opacity=0.6)

alt.LayerChart(...)

In [ ]:
# Running the algorithm
dico={}
r=0
gagnant=[]

for i in range(1,len(liste_factores)):
    # Creating an itertool object with all combinations of factors
    liste=(itertools.combinations(liste_factores,i))
    
    for i in liste:
        # Creating a string with + to integrate factors into model
        element1=' + '.join(list(i))
        
        # Creating string with final parameters for linear regression
        a="totalsales ~ {}".format(element1)
        
        # Running the linear regression
        model = sm.OLS.from_formula(a, data=da)
        result = model.fit()
        
        # Creating a dictionary with all the combinations' adj r squared results
        dico[element1]=result.rsquared_adj.round(3)
        r1=dico[element1]
        
        # saves best adjusted r-squared and liste of columns 
        if r1>r:
            r=r1
            gagnant=element1
        
# Extracting second best model
second=sorted(dico.items(), key=lambda x: x[1],reverse=True)[:5][1][0]

# Running the model
a="totalsales ~ {}".format(gagnant)
model = sm.OLS.from_formula(a, data=da)
result = model.fit()

# Returns best model according to Adjusted R-squared
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             totalsales   R-squared:                       0.389
Model:                            OLS   Adj. R-squared:                  0.295
Method:                 Least Squares   F-statistic:                     4.146
Date:                Wed, 25 Aug 2021   Prob (F-statistic):            0.00994
Time:                        15:09:47   Log-Likelihood:                -386.11
No. Observations:                  31   AIC:                             782.2
Df Residuals:                      26   BIC:                             789.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      9.528e+04   3.81e+04      2.502      0.019     1.7e+04    1.74e+05
gdp           -2.996e-08   9.83e-09     -3.048      0.005   -5.02e-08   -9.76e-09
merchsales     2.194e-07   6.59e-08      3.327      0.003    8.39e-08    3.55e-07
distance         -3.6512      1.903     -1.919      0.066      -7.562       0.259
expend_health -7609.1502   4382.788     -1.736      0.094   -1.66e+04    1399.800
==============================================================================
Omnibus:                       15.540   Durbin-Watson:                   1.872
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               16.704
Skew:                           1.481   Prob(JB):                     0.000236
Kurtosis:                       5.039   Cond. No.                     1.09e+13
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.09e+13. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
# Returns Top 10 models according to adjusted R-squared
ordered_liste=sorted(dico.items(), key=lambda x: x[1],reverse=True)[:10]
ordered_liste

[('gdp + merchsales + distance + expend_health', 0.295),
 ('gdp + merchsales + internet + distance + expend_health', 0.272),
 ('gdp + cpi + merchsales + distance + expend_health', 0.271),
 ('gdp + merchsales + distance + expend_health + ppp_cap', 0.27),
 ('gdp + population + merchsales + distance + expend_health', 0.268),
 ('gdp + cpi + population + merchsales + distance + expend_health', 0.258),
 ('gdp + population + merchsales + internet + distance + expend_health',
  0.253),
 ('gdp + merchsales + distance', 0.25),
 ('gdp + population + merchsales + distance + expend_health + ppp_cap', 0.246),
 ('gdp + merchsales + internet + distance + expend_health + ppp_cap', 0.244)]

In [ ]:
# Running the model
a="totalsales ~ {}".format(second)
model = sm.OLS.from_formula(a, data=da)
result = model.fit()

# Returns best model according to Adjusted R-squared
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             totalsales   R-squared:                       0.393
Model:                            OLS   Adj. R-squared:                  0.272
Method:                 Least Squares   F-statistic:                     3.244
Date:                Wed, 25 Aug 2021   Prob (F-statistic):             0.0215
Time:                        15:37:47   Log-Likelihood:                -386.00
No. Observations:                  31   AIC:                             784.0
Df Residuals:                      25   BIC:                             792.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      7.434e+04   6.41e+04      1.159      0.257   -5.78e+04    2.06e+05
gdp           -2.862e-08   1.05e-08     -2.724      0.012   -5.03e-08   -6.98e-09
merchsales     2.096e-07   7.11e-08      2.948      0.007    6.32e-08    3.56e-07
internet        301.0100    734.892      0.410      0.686   -1212.529    1814.549
distance         -3.4441      1.999     -1.723      0.097      -7.561       0.673
expend_health -7726.0791   4463.800     -1.731      0.096   -1.69e+04    1467.290
==============================================================================
Omnibus:                       15.035   Durbin-Watson:                   1.852
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               15.816
Skew:                           1.455   Prob(JB):                     0.000368
Kurtosis:                       4.942   Cond. No.                     1.81e+13
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.81e+13. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
result.params

Intercept        9.528416e+04
gdp             -2.995532e-08
merchsales       2.193964e-07
distance        -3.651236e+00
expend_health   -7.609150e+03
dtype: float64

In [ ]:
table = pd.pivot_table(df2, values='sales', index=['Code'],
                    columns=['month_year'], aggfunc=np.sum, fill_value=0)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=023a0e03-cc30-4d6d-b448-6d27799cca93' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>